In [1]:
"""This code is for reading simulated SNe waveforms"""

import h5py
import numpy as np
import matplotlib
matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt
from scipy import integrate, interpolate, signal, optimize, stats
import cPickle as pickle

import lal
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Flatten
from keras.optimizers import Nadam, SGD
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
import pyfftw
np.set_printoptions(edgeitems=30, linewidth=160)
import warnings
warnings.filterwarnings('ignore')


Using TensorFlow backend.


In [85]:
SNR_= np.array(waveformfile[u'reduced_data'][u'SNR(aLIGOfrom10kpc)'])
i = 3
print(SNR_[i])
print(SNR[i])

17.2167168764
360.8339166439981


In [3]:
filename = './Data/GWdatabase.h5'
waveformfile = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % waveformfile.keys())

reduced_data = waveformfile.keys()[0]
waveformfilekey = waveformfile.keys()[1]
yeofrho = waveformfile.keys()[2]

waveformfamily = []
waveformfamily_keys = []

for i, key in enumerate(waveformfile[waveformfilekey].keys()):
    waveformfamily.append(key)
    if i == 0:
        for j, _ in enumerate(waveformfile[waveformfilekey][waveformfamily[i]].keys()):
            waveformfamily_keys.append(waveformfile[waveformfilekey][waveformfamily[i]].keys()[j])


Keys: [u'reduced_data', u'waveforms', u'yeofrho']


In [4]:
PctMe = lal.PC_SI 
Dist = 10.0 * 1e3 * PctMe * 1e2 #(centimeters)
findmax = 0
k = 0 
findmin = 1e10
kmin = 0
length = np.zeros(len(waveformfamily))
#waveformfamily = [waveformfamily[0]]
no_waves_considered = 100
for i in range(len(waveformfamily[0:no_waves_considered])):
    waveformnumber = i

    ts = np.array(waveformfile[waveformfilekey][waveformfamily[waveformnumber]][u't-tb(s)']) 
    #waves = np.array(waveforms[waveformkey][waveformfamily[waveformnumber]][u'strain*dist(cm)']) / Dist 
    if findmax < len(ts):
        findmax = len(ts)
        k = i
    if findmin > len(ts):
        findmin = len(ts)
        kmin = i
    length[i] = len(ts)   
    #plt.plot(ts, waves)
#plt.show()
print(findmax, k, findmin, kmin)
findmax += 5

(27818, 94, 13188, 12)


In [17]:
ts = np.array(waveformfile[waveformfilekey][waveformfamily[197]][u't-tb(s)']) 
print(ts[-1], ts[2], ts[3])
print(len(ts))

(1.6566702, 0.0010305176, 0.0010457764)
108507


In [15]:
plt.scatter(np.arange(len(length)), length)
plt.show()


In [5]:
def padandextractwave(waveformfile, waveformfilekey, waveformfamily, strainkey, wavemaxlength, FDish, no_waves_considered):
    noofwaves = len(waveformfamily[0:no_waves_considered])
    extractedwaveforms = np.array([np.zeros(wavemaxlength / 8) for i in range(noofwaves)])
    R = 8
    for i, whichsimulation in enumerate(waveformfamily[0:no_waves_considered]):
        
        wave = np.array(waveformfile[waveformfilekey][whichsimulation][strainkey]) / FDish
        wavelength = len(wave)
        temporary = np.pad(wave, (0, wavemaxlength - wavelength), 'constant', constant_values = 0)
        extractedwaveforms[i] = signal.decimate(temporary, R, ftype='iir')
        
    return extractedwaveforms
    

In [6]:
Sr = 65535

length[k] = 0
#findmax = np.amax(length).astype(int) + 1
findmax = 108512#len(SNewaves[0]) +1
Tobs = findmax / 65535.0
SNewaves = padandextractwave(waveformfile, waveformfilekey, waveformfamily, u'strain*dist(cm)', findmax, Dist, no_waves_considered)


New_sr = (len(SNewaves[0]) - 1) / Tobs
New_dt = 1.0 / New_sr


ts = np.arange(len(SNewaves[0])) * New_dt

In [47]:
len(SNewaves[0]) * New_dt, ts[-1]

(1.6559090652298774, 1.6557869840543222)

In [7]:
# This Part of The Code Reads The Noise Curve Files For The Selected Detectors  
#help(lal.ArrivalTimeDiff)
def ASDtxt(x):
    return {
        'LET':'./ASD/ET_D.txt',
        'LCE':'./ASD/CE.txt',
        'H1': './ASD/ligoII_NS.txt',
        'L1': './ASD/ligoII_NS.txt',
        'V1': './ASD/virgoII.txt',
        'I2': './ASD/ligoII_NS.txt',
        'KAGRA': './ASD/ligoII_NS.txt',
        'ET_1': './ASD/ET_D.txt',
        'ET_2': './ASD/ET_D.txt',
        'ET_3': './ASD/ET_D.txt',
        'A2': './ASD/ligoII_NS.txt',
        'A2.5': './ASD/ligoII_NS.txt',
    }[x]


In [8]:
# reading the noise curves.
def readnos(detector, f_points):
    nos_file = ASDtxt(detector)
    f_str = []
    ASD_str = []
    file = open(nos_file, 'r')
    readFile = file.readlines()
    file.close()
    f = []
    ASD = []
    
    for line in readFile:
        p = line.split()
        f_str.append(float(p[0]))
        ASD_str.append(float(p[1]))
    f = np.log10(np.array(f_str))
    ASD = np.log10(np.array(ASD_str))
    nosinterpolate = interpolate.splrep(f, ASD, w=1.0*np.ones(len(ASD)), s=0)
    
    nos = interpolate.splev(np.log10(f_points), nosinterpolate, der = 0, ext = 3)
    nos = 10**nos
    
    return nos

In [9]:
def noisegenerator(Tobs, det, SR, df, dt):
    Ns = Tobs * SR
    Nf = int(np.ceil(Ns /2) + 1)
    fs = np.arange(Nf) * df
    ASD = readnos('H1', fs)
    Amp = np.sqrt(Tobs) * ASD
    
    real_nos = Amp * np.random.normal(0.0, 1.0, Nf)
    img_nos = Amp * np.random.normal(0.0, 1.0, Nf)
    
    nos = real_nos + 1j * img_nos
    
    idx_1 = int(20 // df)
    nos[0:idx_1] = 0
    
    
    fftinput = pyfftw.empty_aligned(len(nos), dtype='complex128')
    
    fft_object = pyfftw.builders.irfft(fftinput)

    nos_realization = fft_object(nos) * df

    return nos_realization, fs
    

In [10]:
def data_generator(seed, ts, Sr, Det, SNewaves, N_rz, multiplication):
    np.random.seed(seed)
    data = np.array([np.zeros_like(ts) for i in range(N_rz)])
    
    SNR = np.zeros(N_rz)
    Ns = len(ts)
    Nf = int(np.ceil(Ns /2) + 1)
    Tobs = ts[-1]
    df = 1.0/Tobs
    
    fs = np.arange(Nf) * df
    ASD = readnos(Det, fs)
    
    for i in range(N_rz):
        if (i+1) % 1000 == 0:
            msg = 'The %s th to %s th noise realizations are now being generated.' %(i+1, i+1000)
            print(msg)
        data[i], _ = noisegenerator(Tobs, Det, Sr, df, dt)
        
    for i in range(multiplication):
        for j in range(len(SNewaves)):
            
            count = i * len(SNewaves) + j
            if (count + 1) % 1000 == 0 and count < 4999:
                msg = 'The %s th to %s th samples of the data set are now being generated.' %(count + 1,count + 1000)
                print(msg)
            data[count] += SNewaves[j]
            
            fftinput_1 = pyfftw.empty_aligned(len(data[count]), dtype='complex128')
            fft_object_1 = pyfftw.builders.rfft(fftinput_1)
            temporary = fft_object_1(data[count]) * 1.0/New_sr
            temporary = temporary / ASD 
        
        
            SNR[count] = np.sqrt(4.0 * sum(abs(temporary[int(100.0/df) : int(500.0/df)]) ** 2 * df))
        
            fftinput_2 = pyfftw.empty_aligned(len(temporary), dtype='complex128')
            fft_object_2 = pyfftw.builders.irfft(fftinput_2)
            data[count] = fft_object_2(temporary) * df
    
            
    return data, SNR
        


In [ ]:
plt.plot(data[2500])
plt.show()

In [11]:
dt = 1.0/New_sr
multiplication = 25
seed = 10
Det = 'H1'
N_rz = 5000#- len(SNewaves)  * multiplication
data, SNR = data_generator(seed, ts, New_sr, Det, SNewaves, N_rz, multiplication)


The 1000 th to 1999 th noise realizations are now being generated.
The 2000 th to 2999 th noise realizations are now being generated.
The 3000 th to 3999 th noise realizations are now being generated.
The 4000 th to 4999 th noise realizations are now being generated.
The 5000 th to 5999 th noise realizations are now being generated.
The 1000 th to 1999 th samples of the data set are now being generated.
The 2000 th to 2999 th samples of the data set are now being generated.


In [55]:
plt.plot(abs(data[59]))
plt.show()

In [42]:
train_start_index_signal, train_end_index_signal

(0, 1200)

In [43]:
train_start_index_noise, train_end_index_noise

(2500, 3700)

In [44]:
val_start_index_signal, val_end_index_signal

(1200, 2400)

In [47]:
val_start_index_noise, val_end_index_noise

(3700, 4900)

In [45]:
test_start_index_signal, test_end_index_noise

(2400, 5000)

In [12]:
save_for_test = 100
nos_portion = (len(data) - multiplication * len(SNewaves) - save_for_test)/2


nos_start = multiplication * len(SNewaves)

train_start_index_signal = 0
train_end_index_signal = (multiplication * len(SNewaves) - save_for_test) / 2

train_start_index_noise = nos_start
train_end_index_noise = nos_start + nos_portion


val_start_index_signal = train_end_index_signal
val_end_index_signal = multiplication * len(SNewaves) - save_for_test

val_start_index_noise = train_end_index_noise
val_end_index_noise = train_end_index_noise + nos_portion


test_start_index_signal = val_end_index_signal
test_end_index_signal =  val_end_index_signal + save_for_test

test_start_index_noise = val_end_index_noise
test_end_index_noise =  val_end_index_noise + save_for_test


train_signal = np.concatenate((data[train_start_index_signal:train_end_index_signal], 
                               data[train_start_index_noise:train_end_index_noise]))

train_label = np.concatenate((np.ones(train_end_index_signal - train_start_index_signal), 
                              np.zeros(train_end_index_noise - train_start_index_noise)))

val_signal = np.concatenate((data[val_start_index_signal:val_end_index_signal], 
                            data[val_start_index_noise:val_end_index_noise]))

val_label = np.concatenate((np.ones(val_end_index_signal - val_start_index_signal), 
                            np.zeros(val_end_index_noise - val_start_index_noise)))


test_signal = np.concatenate((data[test_start_index_signal : test_end_index_signal], 
                             data[test_start_index_noise : test_end_index_noise]))

test_label = np.concatenate((np.ones(test_end_index_signal - test_start_index_signal), 
                            np.zeros(test_end_index_noise - test_start_index_noise)))








In [13]:
plt.plot(train_signal[0])
print(train_label[0])
plt.show()

1.0


In [51]:
sanity(train_signal, train_label)
#sanity(test_signal, test_label)
#sanity(val_signal, val_label)

1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399


In [50]:
def sanity(signal, label):
    for i, test_signal in enumerate(signal):
        if np.array_equal(test_signal, np.zeros_like(test_signal)) == True and label[i] != 0:
            print(i)
        elif np.array_equal(test_signal, np.zeros_like(test_signal)) == False and label[i] != 1:
            print(i)  
    

In [49]:
pathandname = "./Data/simulatedwave_plus_noise_first_100_fft.pkl"
fp = open(pathandname,"w")
pickle.dump([train_signal, train_label, val_signal, val_label, test_signal, test_label], fp)
fp.close()

In [74]:
with open('./Data/simulatedwave_plus_noise_first_100_fft.pkl') as f:
    train_signal, train_label, val_signal, val_label, test_signal, test_label = pickle.load(f)

In [29]:
train_signal = train_signal * 1e7
val_signal = val_signal * 1e7
test_signal = test_signal * 1e7

In [82]:
i = 1#1834
y = 1784

In [83]:
plt.plot(train_signal[i])
#plt.plot(SNewaves[y])
print(train_label[i])
plt.show()

1.0


In [14]:
i = 0
test = np.fft.rfft(train_signal[i]) * dt
Ns = len(data[i])
Nf = int(np.ceil(Ns /2) + 1)
fs = np.arange(Nf) * df
ASD = readnos('H1', fs)
test = test / ASD 
test = np.fft.irfft(test) * df
plt.plot(test)
plt.show()


In [33]:
np.amin(a)

0.009027880081392462

In [14]:
def shuffle_data(train_signal, train_label, val_signal, val_label, test_signal, test_label,  shuffle_times):
    
    for i in range(shuffle_times):
        state = np.random.randint(0,100)
        train_signal, train_label = shuffle(train_signal, train_label, random_state=state)

        state = np.random.randint(0,100)
        test_signal, test_label = shuffle(test_signal, test_label, random_state=state)

        state = np.random.randint(0,100)
        val_signal, val_label = shuffle(val_signal, val_label, random_state=state)

    return train_signal, train_label, val_signal, val_label, test_signal, test_label

In [15]:
train_signal, train_label, val_signal, val_label, test_signal, test_label = shuffle_data(train_signal, train_label, val_signal, val_label, test_signal, test_label,  4)

In [16]:
i = 14
#plt.plot(ts, train_signal[i])
plt.plot(ts, val_signal[i])
#print(train_label[i])
print(val_label[i])
plt.show()

1.0


In [21]:
"""Below is the CNN part of this code""" 

batch_size = 30      # number of time series per batch
num_classes = 2      # signal or background
epochs = 20          # number of full passes of the dataset
outdir = './results' # directory to store results in



In [22]:
length = len(train_signal)
signal_len = len(train_signal[0])

train_signal = train_signal.reshape(length,1, signal_len)
test_signal = test_signal.reshape(len(test_signal),1, signal_len)
val_signal = val_signal.reshape(length,1, signal_len)

In [23]:
keras.backend.set_image_data_format('channels_first')
signal_len = len(train_signal[0][0])

train_signal = train_signal.reshape(-1, 1, 1, signal_len)
val_signal = val_signal.reshape(-1, 1, 1, signal_len)
test_signal = test_signal.reshape(-1, 1, 1, signal_len)

input_shape = train_signal.shape[1:]

In [24]:
train_label = keras.utils.to_categorical(train_label , num_classes)
val_label = keras.utils.to_categorical(val_label, num_classes)
test_label = keras.utils.to_categorical(test_label, num_classes)

In [ ]:
del model

In [2]:
model = Sequential()    # define the type of keras model

# add the layers
# conv1
model.add(Conv2D(16, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,4)))


model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,4)))


# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))
model.add(Dropout(0.5))
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))
# print a summary
model.summary()

NameError: name 'input_shape' is not defined

In [37]:
model = Sequential()    # define the type of keras model

# add the layers
# conv1
model.add(Conv2D(8, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))
# conv2
model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))
# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))
model.add(Dropout(0.5))
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))
# print a summary
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 8, 1, 13501)       520       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 1, 1687)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 1, 1672)       2064      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 1, 278)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4448)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                142368    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
__________

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr = 0.1),
              metrics=['accuracy'])


In [27]:

modelCheck = ModelCheckpoint('{0}/best_weights.hdf5'.format(outdir), monitor='val_acc', verbose=0, 
                save_best_only=True,save_weights_only=True, mode='auto', period=0)


In [28]:
history = model.fit(train_signal, train_label,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(val_signal, val_label),
                    callbacks = [modelCheck]
                   )

Train on 2400 samples, validate on 2400 samples
Epoch 1/20
2400/2400 [==============================] - 3s 1ms/step - loss: 6.6408 - acc: 0.5058 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/20
2400/2400 [==============================] - 2s 874us/step - loss: 7.8760 - acc: 0.4967 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/20
2400/2400 [==============================] - 2s 899us/step - loss: 7.8902 - acc: 0.4979 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/20
2400/2400 [==============================] - 2s 884us/step - loss: 7.8741 - acc: 0.4992 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/20
2400/2400 [==============================] - 2s 881us/step - loss: 7.8724 - acc: 0.4975 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/20
2400/2400 [==============================] - 2s 882us/step - loss: 7.8493 - acc: 0.4967 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/20
2400/2400 [==============================] - 2s 865us/step - loss: 7.8298 - acc: 0.4988 - val_loss: 8.0590 - val_acc: 0.5000
Ep

In [42]:
# load the best model
model.load_weights('{0}/best_weights.hdf5'.format(outdir))
# evaluate
eval_results = model.evaluate(test_signal, test_label,
                              sample_weight=None,
                              batch_size=batch_size, verbose=1)
print('Test loss:', eval_results[0])
print('Test accuracy:', eval_results[1])
signal_preds = model.predict(test_signal)


200/200 [==============================] - 0s 365us/step
('Test loss:', 1.1920930376163597e-07)
('Test accuracy:', 1.0)


In [43]:
signal_preds = model.predict(test_signal)
print(signal_preds)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 

In [44]:
fig , axs = plt.subplots(2,1, sharex = True)
axs = axs.ravel()
# plot history
axs[0].plot(history.history['loss'], label = 'loss')
axs[0].plot(history.history['val_loss'], label = 'val loss')
axs[1].plot(history.history['acc'], label = 'acc')
axs[1].plot(history.history['val_acc'], label = 'val_acc')
# set labels
axs[0].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Acc')
# legends
axs[0].legend()
axs[1].legend()
# grids
axs[0].grid()
axs[1].grid()
plt.show()